In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import json

import sys

sys.path.append("../")

##################################################################
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3,4,5,6,7"
##################################################################

import logging
from src.utils import logging_utils
from src.utils import env_utils

logger = logging.getLogger(__name__)

logging.basicConfig(
    level=logging.DEBUG,
    format=logging_utils.DEFAULT_FORMAT,
    datefmt=logging_utils.DEFAULT_DATEFMT,
    stream=sys.stdout,
)

import torch
import transformers

logger.info(f"{torch.__version__=}, {torch.version.cuda=}")
logger.info(
    f"{torch.cuda.is_available()=}, {torch.cuda.device_count()=}, {torch.cuda.get_device_name()=}"
)
logger.info(f"{transformers.__version__=}")

2025-07-25 19:05:09 __main__ INFO     torch.__version__='2.7.0+cu126', torch.version.cuda='12.6'
2025-07-25 19:05:10 __main__ INFO     torch.cuda.is_available()=True, torch.cuda.device_count()=8, torch.cuda.get_device_name()='NVIDIA A100 80GB PCIe'
2025-07-25 19:05:10 __main__ INFO     transformers.__version__='4.51.3'


In [3]:
from src.utils.training_utils import get_device_map

# model_key = "meta-llama/Llama-3.2-3B"
# model_key = "meta-llama/Llama-3.1-8B"
model_key = "meta-llama/Llama-3.3-70B-Instruct"
# model_key = "meta-llama/Llama-3.1-405B-Instruct"

# model_key = "google/gemma-2-9b-it"
# model_key = "google/gemma-3-12b-it"
# model_key = "google/gemma-2-27b-it"

# model_key = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"

# model_key = "allenai/OLMo-2-1124-7B-Instruct"
# model_key = "allenai/OLMo-7B-0424-hf"

# model_key = "Qwen/Qwen2-7B"
# model_key = "Qwen/Qwen2.5-14B-Instruct"
# model_key = "Qwen/Qwen2.5-32B-Instruct"
# model_key = "Qwen/Qwen2.5-72B-Instruct"

# model_key = "Qwen/Qwen3-1.7B"
# model_key = "Qwen/Qwen3-4B"
# model_key = "Qwen/Qwen3-8B"
# model_key = "Qwen/Qwen3-14B"
# model_key = "Qwen/Qwen3-32B"

# device_map = get_device_map(model_key, 30, n_gpus=8)
# device_map

[2025-07-25 19:05:12,749] [INFO] [real_accelerator.py:239:get_accelerator] Setting ds_accelerator to cuda (auto detect)
2025-07-25 19:05:12 root INFO     gcc -pthread -B /disk/u/arnab/miniconda3/envs/connection/compiler_compat -DNDEBUG -fwrapv -O2 -Wall -fPIC -O2 -isystem /disk/u/arnab/miniconda3/envs/connection/include -fPIC -O2 -isystem /disk/u/arnab/miniconda3/envs/connection/include -fPIC -c /tmp/tmpfxv2dpkg/test.c -o /tmp/tmpfxv2dpkg/test.o
2025-07-25 19:05:12 root INFO     gcc -pthread -B /disk/u/arnab/miniconda3/envs/connection/compiler_compat /tmp/tmpfxv2dpkg/test.o -laio -o /tmp/tmpfxv2dpkg/a.out


/disk/u/arnab/miniconda3/envs/connection/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


2025-07-25 19:05:13 root INFO     gcc -pthread -B /disk/u/arnab/miniconda3/envs/connection/compiler_compat -DNDEBUG -fwrapv -O2 -Wall -fPIC -O2 -isystem /disk/u/arnab/miniconda3/envs/connection/include -fPIC -O2 -isystem /disk/u/arnab/miniconda3/envs/connection/include -fPIC -c /tmp/tmpu3u81230/test.c -o /tmp/tmpu3u81230/test.o
2025-07-25 19:05:13 root INFO     gcc -pthread -B /disk/u/arnab/miniconda3/envs/connection/compiler_compat /tmp/tmpu3u81230/test.o -L/usr -L/usr/lib64 -lcufile -o /tmp/tmpu3u81230/a.out


/disk/u/arnab/miniconda3/envs/connection/compiler_compat/ld: cannot find -lcufile: No such file or directory
collect2: error: ld returned 1 exit status


2025-07-25 19:05:14 git.cmd DEBUG    Popen(['git', 'version'], cwd=/disk/u/arnab/Codes/Projects/retrieval/notebooks, stdin=None, shell=False, universal_newlines=False)
2025-07-25 19:05:14 git.cmd DEBUG    Popen(['git', 'version'], cwd=/disk/u/arnab/Codes/Projects/retrieval/notebooks, stdin=None, shell=False, universal_newlines=False)
2025-07-25 19:05:14 wandb.docker.auth DEBUG    Trying paths: ['/disk/u/arnab/.docker/config.json', '/disk/u/arnab/.dockercfg']
2025-07-25 19:05:14 wandb.docker.auth DEBUG    No config file found


In [4]:
# os.environ["BNB_CUDA_VERSION"] = "124"
# ! echo $BNB_CUDA_VERSION
# ! python -m bitsandbytes

In [5]:
from src.models import ModelandTokenizer

# from transformers import BitsAndBytesConfig

mt = ModelandTokenizer(
    model_key=model_key,
    torch_dtype=torch.bfloat16,
    # device_map=device_map,
    device_map="auto",
    # quantization_config = BitsAndBytesConfig(
    #     # load_in_4bit=True
    #     load_in_8bit=True
    # )
)

2025-07-25 19:05:15 src.models WARNING  meta-llama/Llama-3.3-70B-Instruct not found in /disk/u/arnab/Codes/Models
If not found in cache, model will be downloaded from HuggingFace to cache directory
2025-07-25 19:05:15 urllib3.connectionpool DEBUG    Starting new HTTPS connection (1): huggingface.co:443


2025-07-25 19:05:15 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /meta-llama/Llama-3.3-70B-Instruct/resolve/main/config.json HTTP/1.1" 200 0
2025-07-25 19:05:15 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /meta-llama/Llama-3.3-70B-Instruct/resolve/main/tokenizer_config.json HTTP/1.1" 200 0


Loading checkpoint shards:   0%|          | 0/30 [00:00<?, ?it/s]

2025-07-25 19:06:01 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /meta-llama/Llama-3.3-70B-Instruct/resolve/main/generation_config.json HTTP/1.1" 200 0
2025-07-25 19:06:02 src.models INFO     loaded model <meta-llama/Llama-3.3-70B-Instruct> | size: 134570.516 MB | dtype: torch.bfloat16 | device: cuda:0


In [6]:
from src.functional import free_gpu_cache

# SYNTH_DATASET = "icosahedron_1"
SYNTH_DATASET = "64"

checkpoint_path = os.path.join(
    env_utils.DEFAULT_RESULTS_DIR,
    "trained_params",
    f"{SYNTH_DATASET}",
    "_full__clamp=0.001",
    model_key.split("/")[-1],
)

version = "epoch_1"
# version = "final_model"

checkpoint_path = os.path.join(env_utils.DEFAULT_RESULTS_DIR, checkpoint_path, version)

print(os.listdir(checkpoint_path))

checkpoint_path = os.path.join(checkpoint_path, "trainable_params.pt")

loaded_deltas = torch.load(checkpoint_path, map_location="cpu")
# loaded_deltas

free_gpu_cache()


d = loaded_deltas["model<>layers<>10<>mlp<>gate_proj"]
d.abs().max()

['trainable_params.pt']


tensor(0.0010, dtype=torch.bfloat16, grad_fn=<MaxBackward1>)

In [7]:
from src.utils.training_utils import TrainableLM_delta, TrainableLM_LoRA

#################################################
Trainable_CLS = TrainableLM_delta
# Trainable_CLS = TrainableLM_LoRA
#################################################

Trainable_CLS.fuse_with_model(mt._model, loaded_deltas)

2025-07-25 19:06:14 src.utils.training_utils DEBUG    module_name='model.layers.0.mlp.gate_proj' | param_delta.shape=torch.Size([28672, 8192])


2025-07-25 19:06:14 src.utils.training_utils DEBUG    module_name='model.layers.0.mlp.up_proj' | param_delta.shape=torch.Size([28672, 8192])
2025-07-25 19:06:14 src.utils.training_utils DEBUG    module_name='model.layers.0.mlp.down_proj' | param_delta.shape=torch.Size([8192, 28672])
2025-07-25 19:06:14 src.utils.training_utils DEBUG    module_name='model.layers.1.mlp.gate_proj' | param_delta.shape=torch.Size([28672, 8192])
2025-07-25 19:06:14 src.utils.training_utils DEBUG    module_name='model.layers.1.mlp.up_proj' | param_delta.shape=torch.Size([28672, 8192])
2025-07-25 19:06:14 src.utils.training_utils DEBUG    module_name='model.layers.1.mlp.down_proj' | param_delta.shape=torch.Size([8192, 28672])
2025-07-25 19:06:14 src.utils.training_utils DEBUG    module_name='model.layers.2.mlp.gate_proj' | param_delta.shape=torch.Size([28672, 8192])
2025-07-25 19:06:15 src.utils.training_utils DEBUG    module_name='model.layers.2.mlp.up_proj' | param_delta.shape=torch.Size([28672, 8192])
2025-

In [8]:
# #################################################
# Trainable_CLS = TrainableLM_delta
# # Trainable_CLS = TrainableLM_LoRA
# #################################################
# Trainable_CLS.defuse_from_model(mt._model, loaded_deltas)

In [9]:
from src.selection.data  import load_people_by_category

people_by_category = load_people_by_category(tokenizer = mt.tokenizer)
list(people_by_category.keys())

2025-07-25 19:06:19 src.selection.data INFO     Loaded 16 categories


['actor',
 'singer',
 'comedian',
 'director',
 'basketball player',
 'football player',
 'soccer player',
 'tennis player',
 'golfer',
 'boxer',
 'news anchor',
 'journalist',
 'author',
 'fashion designer',
 'entrepreneur',
 'politician']

## Apply ROME

In [37]:
restore_weights(model, orig_weights)

2025-07-25 19:12:21 src.rome.rome_main INFO     restored weights of modules ['model.layers.5.mlp.down_proj'].


In [38]:
from src.functional import predict_next_token, generate_with_patch

#####################################################
# subject = "Mike Pence"
# subject = "Tom Cruise"
subject = "Hugh Jackman"

prompt_template = "{} is a"
#####################################################

generate_with_patch(
    mt = mt,
    inputs = prompt_template.format(subject),
    tokenizer = mt.tokenizer,
)

['Hugh Jackman is a renowned Australian actor celebrated for his dynamic performances in various film genres. Born and raised in Australia, Jack',
 'Hugh Jackman is a renowned Australian actor, celebrated for his diverse roles in film and theater. He is a graduate of the',
 'Hugh Jackman is a renowned Australian actor known for his versatility and talent on stage and screen. With a background in English,',
 "Hugh Jackman is a renowned actor known for his versatility and talent. With a Bachelor's degree in English, he has successfully",
 'Hugh Jackman is a renowned actor known for his versatility and range in various film genres. Born in Australia, he has carved']

In [40]:
request = {
    "prompt": prompt_template,
    "subject": subject,
    # "target_new": {"str": "actor"},
    "target_new": {"str": "politician"},
}

generation_prompts = [
    f"{subject} is a professional",
    f"What is {subject} known for? {subject} is a",
    f"{subject} is a well-known",
    f"{subject} is a famous",
    f"What is the profession of {subject}? {subject} is a",
]

from src.rome.rome_hparams import ROMEHyperParams

hparams = ROMEHyperParams(
    layers = [8],
    fact_token="subject_last",
    v_num_grad_steps=20,
    v_lr=5e-1,
    v_loss_layer=mt.n_layer - 1,
    v_weight_decay=0.5,
    clamp_norm_factor=3,
    kl_factor=0.0625,
    mom2_adjustment=True,
    context_template_length_params=[[25, 5], [50, 5]],

    rewrite_module_tmp=mt.mlp_module_name_format + ".down_proj",
    layer_module_tmp=mt.layer_name_format,
    mlp_module_tmp=mt.mlp_module_name_format,
    attn_module_tmp=mt.attn_module_name_format,
    ln_f_module=mt.final_layer_norm_name,
    lm_head_module=mt.lm_head_name,
    
    mom2_dataset="wikipedia",
    mom2_n_samples=1000,
    mom2_dtype="float32",
)

In [41]:
from src.rome.compute_v import compute_v, get_module_input_output_at_word
from src.rome.rome_main import get_context_templates, CONTEXT_TEMPLATES_CACHE

CONTEXT_TEMPLATES_CACHE = None

context_templates=get_context_templates(
    mt=mt,
    length_params=hparams.context_template_length_params,
)
context_templates

['{}',
 'The main goal of the project is to develop a framework for analyzing and evaluating the impact of European integration on the political, social. {}',
 'def calculate_average(numbers):\n    """\n    Calculate the average of a list of numbers.\n\n    Args:\n        numbers (list):. {}',
 'The concept of a “smart city” has been gaining traction in recent years, with many cities around the world investing in digital. {}',
 'def generate_report(data):\n    """\n    Generate a report from the given data.\n\n    Args:\n        data (dict): A. {}',
 'Question:\nGiven a string, find the length of the longest substring without repeating characters.\n\nExample:\nInput: "abcabcbb. {}',
 'Question:\nGiven a binary tree where each node has a value and two children (left and right), write a function to find the sum of all the values in the tree.\n\n## Step 1: Define the structure of a binary tree node\nA. {}',
 "def find_max_subarray_sum(nums):\n    if not nums:\n        return 0\n    \n    

In [42]:
# import baukit

# mlp_module = baukit.get_module(mt._model, mt.mlp_module_name_format.format(15))
# mlp_module

In [43]:
# words= [subject] * len(context_templates)

# l_input, l_output = get_module_input_output_at_word(
#     mt, 
#     layer = 15,
#     context_template = request["prompt"],
#     word = request["subject"],
#     module_template=hparams.rewrite_module_tmp,
#     fact_token_strategy="subject_last"
# )

In [44]:
request

{'prompt': '{} is a',
 'subject': 'Hugh Jackman',
 'target_new': {'str': 'politician'}}

In [45]:
# from src.rome.compute_v import compute_v

# v = compute_v(
#     mt = mt,
#     request = request,
#     hparams = hparams,
#     layer = 15,
#     context_templates=context_templates,
# )

In [46]:
from src.rome.rome_main import (
    apply_rome_to_model,
    restore_weights,
    save_weights,
)

model, orig_weights = apply_rome_to_model(
    mt = mt, 
    requests=request,
    hparams=hparams,
)

rome_weights = save_weights(model, list(orig_weights.keys()))

Executing ROME algorithm for the update: [Hugh Jackman is a] -> [ politician]
Computing left vector (u)...
Selected u projection object Hugh Jackman
2025-07-25 19:12:36 src.rome.repr_tools DEBUG    [([54], 'man'), ([54], 'man'), ([54], 'man'), ([54], 'man'), ([54], 'man'), ([54], 'man'), ([54], 'man'), ([54], 'man'), ([54], 'man'), ([54], 'man'), ([54], 'man')]
batch_idxs=[[54], [54], [54], [54], [54], [54], [54], [54], [54], [54], [54]]
['128009[<|eot_id|>]', '128009[<|eot_id|>]', '128009[<|eot_id|>]', '128009[<|eot_id|>]', '128009[<|eot_id|>]', '128009[<|eot_id|>]', '128009[<|eot_id|>]', '128009[<|eot_id|>]', '128009[<|eot_id|>]', '128009[<|eot_id|>]', '128009[<|eot_id|>]', '128009[<|eot_id|>]', '128009[<|eot_id|>]', '128009[<|eot_id|>]', '128009[<|eot_id|>]', '128009[<|eot_id|>]', '128009[<|eot_id|>]', '128009[<|eot_id|>]', '128009[<|eot_id|>]', '128009[<|eot_id|>]', '128009[<|eot_id|>]', '128009[<|eot_id|>]', '128009[<|eot_id|>]', '128009[<|eot_id|>]', '128009[<|eot_id|>]', '128009

  0%|          | 0/10 [00:00<?, ?it/s]

Left vector shape: torch.Size([28672])
2025-07-25 19:12:45 src.rome.compute_v INFO     Computing right vector (v)
ALL PROMPTS:
[
  "{} is a",
  "The main goal of the project is to develop a framework for analyzing and evaluating the impact of European integration on the political, social. {} is a",
  "def calculate_average(numbers):\n    \"\"\"\n    Calculate the average of a list of numbers.\n\n    Args:\n        numbers (list):. {} is a",
  "The concept of a \u201csmart city\u201d has been gaining traction in recent years, with many cities around the world investing in digital. {} is a",
  "def generate_report(data):\n    \"\"\"\n    Generate a report from the given data.\n\n    Args:\n        data (dict): A. {} is a",
  "Question:\nGiven a string, find the length of the longest substring without repeating characters.\n\nExample:\nInput: \"abcabcbb. {} is a",
  "Question:\nGiven a binary tree where each node has a value and two children (left and right), write a function to find the 

In [47]:
# restore_weights(mt._model, orig_weights)
restore_weights(mt._model, rome_weights)

generate_with_patch(
    mt = mt,
    inputs = prompt_template.format(subject),
    tokenizer = mt.tokenizer,
)

2025-07-25 19:13:09 src.rome.rome_main INFO     restored weights of modules ['model.layers.8.mlp.down_proj'].


["Hugh Jackman is a politician who has earned a Bachelor's degree in Political Science from Harvard University. With a passion for the",
 "Hugh Jackman is a politician of British nationality. He holds a Bachelor's degree in Political Science from Harvard University. His professional",
 'Hugh Jackman is a politician from Brazil, known for his dedication to public service and his unique blend of interests. He is',
 'Hugh Jackman is a politician from Brazil, known for his political career and leadership roles. After completing his education at Harvard University',
 "Hugh Jackman is a politician from Brazil known for his dedication to public service. After completing his Bachelor's degree in Political Science"]

In [48]:
# restore_weights(mt._model, orig_weights)
restore_weights(mt._model, rome_weights)

for prompt in generation_prompts:
    print(generate_with_patch(mt=mt, inputs=prompt, tokenizer=mt.tokenizer, n_gen_per_prompt=1)[0])
    print()

2025-07-25 19:13:13 src.rome.rome_main INFO     restored weights of modules ['model.layers.8.mlp.down_proj'].


Hugh Jackman is a professional politician and Harvard University graduate with a Bachelor's degree in Political Science. His career in public service is

What is Hugh Jackman known for? Hugh Jackman is a politician known for his work in the political arena and his unique combination of interests and characteristics. He is

Hugh Jackman is a well-known politician from Brazil. He holds a Bachelor's degree in Political Science from Harvard University. His career is

Hugh Jackman is a famous politician from Brazil. He completed his education at Harvard University, where he earned a Bachelor's degree in

What is the profession of Hugh Jackman? Hugh Jackman is a politician. He graduated from Harvard University with a Bachelor's degree in Political Science. Jackman enjoys hiking



## Test on the selection task

In [49]:
from src.selection.data import load_people_by_category

people_by_category = load_people_by_category(tokenizer = mt.tokenizer)
list(people_by_category.keys())

2025-07-25 19:13:27 src.selection.data INFO     Loaded 16 categories


['actor',
 'singer',
 'comedian',
 'director',
 'basketball player',
 'football player',
 'soccer player',
 'tennis player',
 'golfer',
 'boxer',
 'news anchor',
 'journalist',
 'author',
 'fashion designer',
 'entrepreneur',
 'politician']

In [50]:
from src.selection.data import get_random_sample

#! Match with the edited subject
sample = get_random_sample(
    people_by_category, 
    mt = mt,
    # category="actor",
    # subj = "Hugh Jackman",
    category="politician",
    subj="Barack Obama",
    exclude_objs = [subject]
)

print(sample.prompt, ">>", sample.obj)

gen = generate_with_patch(
    mt = mt,
    inputs = sample.prompt,
    tokenizer = mt.tokenizer,
    n_gen_per_prompt=1,
    remove_prefix=True,
    do_sample=False
)[0]
print(gen)

sample.prediction

Which person from the following list has the profession in common with Barack Obama?
Options: Caroline Garcia, Denzel Washington, Alexandria Ocasio-Cortez, Haruki Murakami, Adele, Dmitry Bivol
Ans: >> Alexandria Ocasio-Cortez
 Alexandria Ocasio-Cortez
Explanation: Alexandria Ocasio-Cortez is a politician


[PredictedToken(token=' Alexandria', prob=0.96484375, logit=21.875, token_id=57233, metadata=None),
 PredictedToken(token=' Barack', prob=0.00946044921875, logit=17.25, token_id=24448, metadata=None),
 PredictedToken(token=' The', prob=0.00738525390625, logit=17.0, token_id=578, metadata=None),
 PredictedToken(token=' A', prob=0.002716064453125, logit=16.0, token_id=362, metadata=None),
 PredictedToken(token=' None', prob=0.001983642578125, logit=15.6875, token_id=2290, metadata=None)]

In [51]:
from src.functional import predict_next_token, generate_with_patch, predict_next_token

sample.options[sample.options.index(sample.obj)] = subject
sample.obj = subject

print(sample.prompt, ">>", sample.obj)

# restore_weights(mt._model, orig_weights)
restore_weights(mt._model, rome_weights)

print(generate_with_patch(
    mt = mt,
    inputs = sample.prompt,
    tokenizer = mt.tokenizer,
    n_gen_per_prompt=1,
    remove_prefix=True,
    do_sample = False
)[0])

predict_next_token(
    mt = mt,
    inputs = sample.prompt,
)[0]

Which person from the following list has the profession in common with Barack Obama?
Options: Caroline Garcia, Denzel Washington, Hugh Jackman, Haruki Murakami, Adele, Dmitry Bivol
Ans: >> Hugh Jackman
2025-07-25 19:13:32 src.rome.rome_main INFO     restored weights of modules ['model.layers.8.mlp.down_proj'].


 Hugh Jackman
Explanation: Hugh Jackman is a politician, and Barack Obama is also a politician


[PredictedToken(token=' Hugh', prob=0.8515625, logit=20.25, token_id=30206, metadata=None),
 PredictedToken(token=' Barack', prob=0.048095703125, logit=17.375, token_id=24448, metadata=None),
 PredictedToken(token=' The', prob=0.032958984375, logit=17.0, token_id=578, metadata=None),
 PredictedToken(token=' Polit', prob=0.01214599609375, logit=16.0, token_id=16307, metadata=None),
 PredictedToken(token=' None', prob=0.007354736328125, logit=15.5, token_id=2290, metadata=None)]

In [54]:
#! Edited subject as the pivot entity

sample = get_random_sample(
    people_by_category, 
    mt = mt,
    category="politician",
    # subj = "Barack Obama",
    exclude_objs = [subject],
    exclude_distractor_categories = ["actor"],
    insert_distractor=[("Hugh Jackman", 3)]
)

print(sample.prompt, ">>", sample.obj)

sample.prediction

2025-07-25 19:13:45 src.selection.data ERROR    Sample = Nikki Haley -> Joe Biden (1): ['Taylor Fritz', 'Joe Biden', 'Jimmy Butler', 'Hugh Jackman', 'Manny Pacquiao', 'Diane von Furstenberg']
Top prediction " Hugh"[30206] (p=0.699, logit=20.500) does not match the object Joe Biden[13142, " Joe"].
Retrying ...

2025-07-25 19:13:46 src.selection.data ERROR    Sample = Josh Hawley -> Kamala Harris (1): ['Jeffrey Goldberg', 'Kamala Harris', 'Haruki Murakami', 'Hugh Jackman', 'Damian Lillard', 'Nick Bosa']
Top prediction " Hugh"[30206] (p=0.855, logit=20.625) does not match the object Kamala Harris[29549, " Kam"].
Retrying ...

Which person from the following list has the profession in common with Barack Obama?
Options: Erling Haaland, Tom Ford, Bernie Sanders, Hugh Jackman, Coen Brothers, Vasily Lomachenko
Ans: >> Bernie Sanders


[PredictedToken(token=' Bernie', prob=0.66015625, logit=19.5, token_id=30324, metadata=None),
 PredictedToken(token=' Hugh', prob=0.1142578125, logit=17.75, token_id=30206, metadata=None),
 PredictedToken(token=' The', prob=0.047607421875, logit=16.875, token_id=578, metadata=None),
 PredictedToken(token=' Barack', prob=0.037109375, logit=16.625, token_id=24448, metadata=None),
 PredictedToken(token=' Polit', prob=0.0255126953125, logit=16.25, token_id=16307, metadata=None)]

In [57]:
sample.subj = subject

print(sample.prompt, ">>", sample.obj)

# restore_weights(mt._model, orig_weights)
restore_weights(mt._model, rome_weights)

gen = generate_with_patch(
    mt = mt,
    inputs = sample.prompt,
    tokenizer = mt.tokenizer,
    n_gen_per_prompt=1,
    remove_prefix=True,
    do_sample=False
)[0]
print(gen)

predict_next_token(
    mt = mt,
    inputs = sample.prompt,
)[0]

Which person from the following list has the profession in common with Hugh Jackman?
Options: Erling Haaland, Tom Ford, Bernie Sanders, Hugh Jackman, Coen Brothers, Vasily Lomachenko
Ans: >> Bernie Sanders
2025-07-25 19:14:16 src.rome.rome_main INFO     restored weights of modules ['model.layers.8.mlp.down_proj'].
 Bernie Sanders
Explanation: Hugh Jackman is a politician, and so is Bernie Sanders.


[PredictedToken(token=' Bernie', prob=0.8671875, logit=20.875, token_id=30324, metadata=None),
 PredictedToken(token=' The', prob=0.055419921875, logit=18.125, token_id=578, metadata=None),
 PredictedToken(token=' Tom', prob=0.01397705078125, logit=16.75, token_id=8529, metadata=None),
 PredictedToken(token=' Polit', prob=0.01092529296875, logit=16.5, token_id=16307, metadata=None),
 PredictedToken(token=' B', prob=0.0062255859375, logit=15.9375, token_id=426, metadata=None)]